# Crawling

In [28]:
import urllib.parse
import time
import requests
from bs4 import BeautifulSoup
from collections import deque
from queue import PriorityQueue
from urllib.parse import urlparse, urljoin
import re
import math
from nltk.stem import PorterStemmer
import concurrent.futures

# URL Canonicalization

In [29]:
def canonicalize_url(url, base_url=None):
        parsed_url = urllib.parse.urlparse(url)
 
        # Rule 1: Convert scheme and host to lower case
        parsed_url = parsed_url._replace(scheme=parsed_url.scheme.lower(), netloc=parsed_url.netloc.lower())
 
        # Rule 2: Remove default ports for http and https
        if parsed_url.port in (80, 443) and parsed_url.scheme in ('http', 'https'):
            parsed_url = parsed_url._replace(netloc=parsed_url.netloc.split(':')[0])
 
        # Rule 3: Make relative URLs absolute (if a base URL is provided)
        if base_url:
            parsed_base_url = urllib.parse.urlparse(base_url)
            if not parsed_url.netloc:
                parsed_url = parsed_url._replace(netloc=parsed_base_url.netloc)
                if not parsed_url.scheme:
                    parsed_url = parsed_url._replace(scheme=parsed_base_url.scheme)
                parsed_url = parsed_url._replace(path=urllib.parse.urljoin(parsed_base_url.path, parsed_url.path))
 
        # Rule 4: Remove fragment
        parsed_url = parsed_url._replace(fragment='')
 
        # Rule 5: Remove duplicate slashes
        clean_path = parsed_url.path
        while '//' in clean_path:
            clean_path = clean_path.replace('//', '/')
        parsed_url = parsed_url._replace(path=clean_path)
        
        # Extra normalizing to avoid unnecessary tokens
        extensions_to_avoid = [".jpg", ".jpeg", ".gif", ".png", ".mp4", ".pdf", "none", ".php", ".ppt", ".doc"]
        for ext in extensions_to_avoid:
            if parsed_url.path.endswith(ext):
                return None
 
        return urllib.parse.urlunparse(parsed_url)

In [30]:
porter_stemmer = PorterStemmer()
keywords = ["cyclone", "storm", "hurricane", "weather", "meterological", "meteor", "ike", "tropical"]
stemmed_keywords = [porter_stemmer.stem(word) for word in keywords]
print(stemmed_keywords)
url_special_chars_regex = r'[-_./~%?&:]'
trusted_domains = {".gov", ".edu", ".org"}

['cyclon', 'storm', 'hurrican', 'weather', 'meterolog', 'meteor', 'ike', 'tropic']


# Fetch page content from web

In [31]:
def fetch_page_content(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            content_type = response.headers.get('content-type', '')
            if content_type.startswith('text/html') or '<html>' in response.text.lower():
                soup = BeautifulSoup(response.content, 'html.parser')
                lang_attr = soup.html.get('lang')
                if lang_attr is None or 'en' in lang_attr.lower():
                    return response.text
                else:
                    print(f"{url} Content is not in English")
                    return None
            else:
                print(f"{url} Content is not HTML")
                return None
        else:
            print("Response status code:", response.status_code)
    except Exception as e:
        print("Error fetching page:", e)
    return None

In [32]:
def parse_links(html_content, base_url):
    soup = BeautifulSoup(html_content, 'html.parser')
    links = set()
    for link in soup.find_all('a', href=True):
        absolute_url = urljoin(base_url, link['href'])
        absolute_url = canonicalize_url(absolute_url, base_url)
        if absolute_url:
            anchor_text = link.get_text().strip()
            links.add((absolute_url, anchor_text))
    return links

In [51]:
def extract_text(html_content):
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        if soup.title and soup.title.string:
            title = soup.title.string.strip() if soup.title else ""
        else:
            title = ""
        body_text = " ".join(paragraph.get_text() for paragraph in soup.find_all('p'))
        body_text = re.sub(r'\s+', ' ', body_text.strip())
        return title, body_text
    except Exception as e:
        print("An error occurred:", e)
        return "", ""

In [52]:
def write_to_corpus_file(url, title, text, file_number):
    docno = url
    file_name = f"hurricane_data_{file_number}.txt"
    try:
        with open(file_name, 'a', encoding='utf-8') as f:
            f.write("<DOC>\n")
            f.write(f"<DOCNO>{docno}</DOCNO>\n")
            f.write(f"<HEAD>{title}</HEAD>\n")
            f.write(f"<TEXT>{text}</TEXT>\n")
            f.write("</DOC>\n")
    except Exception as e:
        print("An error occurred:", e)

In [53]:
def process_document(url, page_content, file_number):
    html_content = page_content
    if html_content:
        title, text = extract_text(html_content)
        write_to_corpus_file(url, title, text, file_number)

# Calculate outlinks score

In [36]:
def scoring_outlinks(url, anchor_text, hurricane_data, wave_number):
    score = 0
    parsed_url = urlparse(url)
    domain = parsed_url.netloc.lower()
    path = parsed_url.path.lower()
    
    stemmed_path = [porter_stemmer.stem(word) for word in re.split(url_special_chars_regex, path.lower())]
    
    is_trusted_domain = any(domain.endswith(ext) for ext in trusted_domains)

    related_words_in_url = sum(keyword in stemmed_path for keyword in stemmed_keywords)

    stemmed_anchor_words = [porter_stemmer.stem(word) for word in anchor_text.lower().split()]

    related_words_in_anchor = sum(keyword in stemmed_anchor_words for keyword in stemmed_keywords)

    total_words_in_url = len(stemmed_path)
    total_words_in_anchor = len(stemmed_anchor_words)

    score += ((related_words_in_anchor + related_words_in_url) / (total_words_in_url + total_words_in_anchor)) * 75
    if 'data' in url or 'data' in anchor_text:
        score += 10
    if 'structure' in url or 'structure' in anchor_text:
        score += 10
    
    if url in hurricane_data:
        in_links_count = sum(hurricane_data[url]['in_links'].values())
        score += in_links_count * 5
        trusted_inlinks_count = sum(1 for inlink, count in hurricane_data[url]['in_links'].items() if urlparse(inlink).netloc.lower().endswith(tuple(trusted_domains)))
        score += trusted_inlinks_count * 3

    if is_trusted_domain:
        score += 30

    score += (15 - wave_number)
    return score

In [37]:
def fetch_robots_txt(url):
    try:
        parsed_url = urlparse(url)
        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
        robots_url = urljoin(base_url, "/robots.txt")
        response = requests.get(robots_url)
        if response.status_code == 200:
            return response.text
    except Exception as e:
        print("Error fetching robots.txt:", e)
    return None

In [38]:
def parse_robots_txt(robots_txt):
    disallowed_urls = set()
    lines = robots_txt.split("\n")
    user_agent = None
    for line in lines:
        if line.lower().startswith("user-agent:"):
            user_agent = line.split(":")[1].strip()
        elif line.lower().startswith("disallow:") and user_agent == "*":
            disallowed_urls.add(line.split(":")[1].strip())
    return disallowed_urls

In [39]:
seed_urls = [
    "http://www.nhc.noaa.gov/outreach/history/",
    "https://en.wikipedia.org/wiki/List_of_United_States_hurricanes",
    "http://en.wikipedia.org/wiki/Hurricane_Ike",
    "https://www.weather.gov/hgx/projects_ike08",
    "https://geology.com/hurricanes/largest-hurricane/"
]

In [43]:
initial_score = -10000
hurricane_data = {}
visited_links = set()
doc_ready_to_write = set()
wave_number = 0
doc_count = 0
write_count = 5000
file_number = 1
num_urls_to_fetch = 30000
urls_in_frontier = set()

frontier_queue = PriorityQueue()

for url in seed_urls:
    frontier_queue.put((initial_score, url, wave_number))

robots_cache = {}

while not frontier_queue.empty() and len(visited_links) < num_urls_to_fetch:
    score, url, wave_number = frontier_queue.get()
    wave_number += 1
    canonicalized_url = canonicalize_url(url)
    canonicalized_url_http = canonicalized_url.replace("https://", "http://")
    canonicalized_url_https = canonicalized_url.replace("http://", "https://")
    if canonicalized_url_http not in visited_links and canonicalized_url_https not in visited_links and canonicalized_url is not None:
        parsed_url = urlparse(canonicalized_url)
        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
        if base_url not in robots_cache:
            robots_txt = fetch_robots_txt(base_url)
            if robots_txt:
                disallowed_urls = parse_robots_txt(robots_txt)
                robots_cache[base_url] = disallowed_urls
            else:
                robots_cache[base_url] = set()
        if parsed_url.path not in robots_cache[base_url]:
            page_content = fetch_page_content(canonicalized_url)
            if page_content is not None:
                links = parse_links(page_content, base_url)
                if canonicalized_url in hurricane_data:
                    hurricane_data[canonicalized_url]['content'] = page_content
                    hurricane_data[canonicalized_url]['out_links'] = links
                else:
                    hurricane_data[canonicalized_url] = {
                        'content': page_content,
                        'in_links': {},
                        'out_links': links
                    }
                visited_links.add(canonicalized_url)
                doc_ready_to_write.add(canonicalized_url)
                for link, anchor_text in links:
                    if link not in hurricane_data:
                        hurricane_data[link] = {
                            'content': None,
                            'in_links': {canonicalized_url: 1},
                            'out_links': set()
                        }
                    else:
                        if canonicalized_url != link:
                            if canonicalized_url in hurricane_data[link]['in_links']:
                                hurricane_data[link]['in_links'][canonicalized_url] += 1
                            else:
                                hurricane_data[link]['in_links'][canonicalized_url] = 1
                    score = scoring_outlinks(link, anchor_text, hurricane_data, wave_number) * -1
                    if link not in urls_in_frontier:
                        frontier_queue.put((score, link, wave_number))
                        urls_in_frontier.add(link)
                doc_count += 1
                if doc_count % 100 == 0:
                    print(f"{doc_count} documents")
            time.sleep(1)
        else:
            print(f"Skipping {canonicalized_url} due to robots.txt rules")
        

print("Crawling completed")

100 documents
200 documents
https://fr.wikipedia.org/wiki/Cyclone_tropical Content is not in English
https://ia.wikipedia.org/wiki/Cyclon_tropic Content is not in English
https://om.wikipedia.org/wiki/Tropical_cyclone Content is not in English
https://et.wiktionary.org/wiki/tropical_cyclone Content is not in English
https://fr.wiktionary.org/wiki/cyclone_tropical Content is not in English
https://ta.wiktionary.org/wiki/tropical_cyclone Content is not in English
https://zh-min-nan.wiktionary.org/wiki/tropical_cyclone Content is not in English
https://zh.wiktionary.org/wiki/tropical_cyclone Content is not in English
300 documents
400 documents
https://et.wiktionary.org/wiki/tropical_storm Content is not in English
https://mg.wiktionary.org/wiki/cyclone_tropical Content is not in English
https://mg.wiktionary.org/wiki/tropical_storm Content is not in English
https://sv.wiktionary.org/wiki/cyclone_tropical Content is not in English
https://ta.wiktionary.org/wiki/tropical_cyclones Content i

1800 documents
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 403
1900 documents
2000 documents
2100 documents
Response status code: 404
Response status code: 404
2200 documents
https://fr.wikipedia.org/wiki/Cyclone_tropical_Alberto Content is not in English
https://fr.wikipedia.org/wiki/Cyclone_tropical_Claudette Content is not in English
https://fr.wikipedia.org/wiki/Cyclone_tropical_Talas Content is not in English
https://it.wikipedia.org/wiki/Weather_(Tycho) Content is not in English
https://mg.wiktionary.org/wiki/ex-trop

Response status code: 404
Response status code: 404
Response status code: 404
Error fetching page: No connection adapters were found for 'ftp://ftp.met.fsu.edu/pub/weather/tropical/Outlook-P/2008052717.ABPZ20'
Error fetching page: No connection adapters were found for 'ftp://ftp.met.fsu.edu/pub/weather/tropical/Australia/2010010419.WTAUT'
3300 documents
Response status code: 503
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Response status code: 404
Error fetching page: HTTPConnectionPool(host='www.srh.noaa.gov', port=80): Max retries exceeded with url: /data/warn_archive/TBW/PSH/0906_191640.txt (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001C461605D90>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
https://cs.w

Error fetching page: HTTPSConnectionPool(host='www.webcitation.org', port=443): Max retries exceeded with url: /6kUm9MuAy?url=http://gwydir.demon.co.uk/advisories/WTPQ20-RJTD_201609131200.htm (Caused by SSLError(CertificateError("hostname 'www.webcitation.org' doesn't match 'webcitation.org'")))
Response status code: 404
Response status code: 404
Response status code: 404
3500 documents
http://tgftp.nws.noaa.gov/data/raw/wt/wtpn21.pgtw..txt Content is not HTML
Error fetching page: No connection adapters were found for 'ftp://ftp.met.fsu.edu/pub/weather/tropical/Atl-Dis/2008071417.AXNT20'
Error fetching page: No connection adapters were found for 'ftp://ftp.met.fsu.edu/pub/weather/tropical/Atl-Dis/2008071512.AXNT20'
Error fetching page: No connection adapters were found for 'ftp://ftp.met.fsu.edu/pub/weather/tropical/Atl-Dis/2008071523.AXNT20'
Error fetching page: No connection adapters were found for 'ftp://ftp.met.fsu.edu/pub/weather/tropical/Atl-Dis/2008071618.AXNT20'
Error fetching 

Error fetching page: No connection adapters were found for 'ftp://ftp.met.fsu.edu/pub/weather/tropical/Outlook-A/2006060608.ABNT20'
Error fetching page: No connection adapters were found for 'ftp://ftp.met.fsu.edu/pub/weather/tropical/Outlook-A/2006060815.ABNT20'
Error fetching page: No connection adapters were found for 'ftp://ftp.met.fsu.edu/pub/weather/tropical/Outlook-A/2006060915.ABNT20'
Error fetching page: No connection adapters were found for 'ftp://ftp.met.fsu.edu/pub/weather/tropical/Outlook-A/2006080602.ABNT20'
Error fetching page: No connection adapters were found for 'ftp://ftp.met.fsu.edu/pub/weather/tropical/Outlook-A/2006080609.ABNT20'
Error fetching page: No connection adapters were found for 'ftp://ftp.met.fsu.edu/pub/weather/tropical/Outlook-A/2006082115.ABNT20'
Error fetching page: No connection adapters were found for 'ftp://ftp.met.fsu.edu/pub/weather/tropical/Outlook-A/2007072909.ABNT20'
Error fetching page: No connection adapters were found for 'ftp://ftp.met.fs

...

Response status code: 403
Error fetching page: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20110611115408/http:/www.atl.ec.gc.ca/weather/hurricane/bulletins/20100905175427.Earl.txt.en (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C6524FD890>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
Error fetching page: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20110611123520/http:/www.atl.ec.gc.ca/weather/hurricane/bulletins/20080722180403.Cristobal.txt.en (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C6524FC710>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
Error fetching page: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded 

Response status code: 404
30000 documents
An error occurred: 'NoneType' object has no attribute 'strip'
Error fetching page: HTTPSConnectionPool(host='www.essl.org', port=443): Max retries exceeded with url: /cms/european-severe-weather-database/ewob/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C60252D1D0>, 'Connection to www.essl.org timed out. (connect timeout=None)'))
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
Error fetching page: HTTPSConnectionPool(host='www.essl.org', port=443): Max retries exce

An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
Response status code: 404
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
Error fetching page: HTTPSConnectionPool(host='www.essl.org', port=443): Max retries exceeded with url: /cms/european-severe-weather-database/ewob/ewob-ads/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001C64BE23850>, 'Connection to www.essl.org timed out. (connect timeout=None)'))
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
Error fetching page: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20160310124443/https:/weather.com/storms/hurri

An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error occurred: 'NoneType' object has no attribute 'strip'
An error

PermissionError: [Errno 13] Permission denied: 'hurricane_data_1.txt'

In [44]:
frontier_length = frontier_queue.qsize()
print("Length of frontier_queue:", frontier_length)

Length of frontier_queue: 675153


In [45]:
visited_url_temp = visited_links

In [46]:
hurricane_data_temp = hurricane_data

In [47]:
frontier_temp = frontier_queue

# Write the crawled data into files

In [ ]:
import json

def dump_inlinks_to_json(hurricane_data, output_file, doc_ready_to_write):
    inlinks_data = {}
    
    for link, data in hurricane_data.items():
        inlinks = list(data.get('in_links', {}).keys())
        if link in doc_ready_to_write:
            inlinks_data[link] = inlinks
    
    with open(output_file, 'w') as json_file:
        json.dump(inlinks_data, json_file, indent=4)

def dump_outlinks_to_json(hurricane_data, output_file, doc_ready_to_write):
    outlinks_data = {}
    
    for link, data in hurricane_data.items():
        outlinks = [sublist[0] for sublist in data.get('out_links', [])]
        if link in doc_ready_to_write: 
            outlinks_data[link] = outlinks
    
    with open(output_file, 'w') as json_file:
        json.dump(outlinks_data, json_file, indent=4)
        
        
inlinks_output_file = 'hurricane_inlinks_final.json'
outlinks_output_file = 'hurricane_outlinks_final.json'


dump_inlinks_to_json(hurricane_data, inlinks_output_file, doc_ready_to_write)
dump_outlinks_to_json(hurricane_data, outlinks_output_file, doc_ready_to_write)
print("File write completed")

In [54]:
file_number = "final"

count_processed = 0
for i, url in enumerate(doc_ready_to_write, start=1):
    if hurricane_data[url]['content'] is not None:
        process_document(url, hurricane_data[url]['content'], file_number)
        count_processed += 1
    if i % 500 == 0:
        print("Processed ", i, " URLs")

print("Total number of documents processed:", count_processed)

Processed  500  URLs
Processed  1000  URLs
Processed  1500  URLs
Processed  2000  URLs
Processed  2500  URLs
Processed  3000  URLs
Processed  3500  URLs
Processed  4000  URLs
Processed  4500  URLs
Processed  5000  URLs
Processed  5500  URLs
Processed  6000  URLs
Processed  6500  URLs
Processed  7000  URLs
Processed  7500  URLs
Processed  8000  URLs
Processed  8500  URLs
Processed  9000  URLs
Processed  9500  URLs
Processed  10000  URLs
Processed  10500  URLs
Processed  11000  URLs
Processed  11500  URLs
Processed  12000  URLs
Processed  12500  URLs
Processed  13000  URLs
Processed  13500  URLs
Processed  14000  URLs
Processed  14500  URLs
Processed  15000  URLs
Processed  15500  URLs
Processed  16000  URLs
Processed  16500  URLs
Processed  17000  URLs
Processed  17500  URLs
Processed  18000  URLs
Processed  18500  URLs
Processed  19000  URLs
Processed  19500  URLs
Processed  20000  URLs
Processed  20500  URLs
Processed  21000  URLs
Processed  21500  URLs
Processed  22000  URLs
Processe

In [55]:
print(wave_number)

8


In [56]:
print(len(visited_links))
print(len(doc_ready_to_write))

30000
30000
